In [1]:
import pandas as pd

from superfv import OutputLoader

In [2]:
def build_df(sim, cats, exclude_first=True):

    slc = slice(2, None) if exclude_first else slice(1, None)

    def get_total_attr(sim, cat, attr):
        times = [
            getattr(timer.data[cat], attr)
            for timer in sim.minisnapshots["stepper_timer"][slc]
        ]
        return sum(times)

    take_step = get_total_attr(sim, "take_step", "cum_time")

    data = []
    for cat in cats:
        data.append(
            dict(
                cat=cat,
                tot_time=get_total_attr(sim, cat, "cum_time"),
                per_time=get_total_attr(sim, cat, "cum_time") / take_step * 100,
                n_calls=get_total_attr(sim, cat, "n_calls"),
            )
        )

    return pd.DataFrame(data)

In [3]:
base_path = "/scratch/gpfs/jp7427/out/timing-of-2d-sine-wave/cupy/"

sim0 = OutputLoader(f"{base_path}p7/GL/N_1024")
sim1 = OutputLoader(f"{base_path}ZS7/N_1024")
# sim = OutputLoader(f"{base_path}MM7/3revs/no_delta/rtol_1e-5/N_2048")

Successfully read simulation output from "/scratch/gpfs/jp7427/out/timing-of-2d-sine-wave/cupy/p7/GL/N_1024"
Successfully read simulation output from "/scratch/gpfs/jp7427/out/timing-of-2d-sine-wave/cupy/ZS7/N_1024"


In [4]:
cats = [
    # "compute_dt",
    # "apply_bc",
    "update_workspaces",
    "update_workspaces:ucc",
    "update_workspaces:wp",
    "update_workspaces:shock_detector",
    "interpolate_faces",
    "interpolate_faces:GL",
    # "interpolate_faces:transverse",
    # "reconstruct_muscl_faces",
    "zhang_shu_limiter",
    "integrate_fluxes",
    "integrate_fluxes:fallback",
    "integrate_fluxes:riemann_solver",
    "integrate_fluxes:GL",
    # "integrate_fluxes:transverse",
    # "MOOD_loop",
    # "compute_RHS",
    "slope_limiter:detect_smooth_extrema",
    # "MOOD_loop:compute_fallback_fluxes",
    # "MOOD_loop:detect_troubled_cells",
    # "MOOD_loop:revise_fluxes",
]

df0 = build_df(sim0, cats)
df1 = build_df(sim1, cats)

In [5]:
df0

,cat,tot_time,per_time,n_calls
0,update_workspaces,0.040329,6.247227,28
1,update_workspaces:ucc,0.010910,1.690066,28
2,update_workspaces:wp,0.011012,1.705899,28
3,update_workspaces:shock_detector,0.000000,0.000000,0
4,interpolate_faces,0.183605,28.441653,54
5,interpolate_faces:GL,0.097269,15.067703,54
6,zhang_shu_limiter,0.000000,0.000000,0
7,integrate_fluxes,0.395004,61.188882,54
8,integrate_fluxes:fallback,0.333867,51.718365,54
9,integrate_fluxes:riemann_solver,0.040266,6.237495,54


In [6]:
df1

,cat,tot_time,per_time,n_calls
0,update_workspaces,0.067466,9.174255,28
1,update_workspaces:ucc,0.010329,1.404508,28
2,update_workspaces:wp,0.010418,1.416646,28
3,update_workspaces:shock_detector,0.015045,2.045856,28
4,interpolate_faces,0.179122,24.357483,54
5,interpolate_faces:GL,0.091547,12.448777,54
6,zhang_shu_limiter,0.310757,42.257670,27
7,integrate_fluxes,0.150081,20.408408,54
8,integrate_fluxes:fallback,0.088828,12.079058,54
9,integrate_fluxes:riemann_solver,0.040555,5.514779,54
